# Step 1: Convert a Book to Markdown
In this notebook, a book is converted to markdown format. The book can be in either PDF or EPUB format. The conversion is done using the custom `textProcessing` module.

In [ ]:
# Import the custom module
import os
import textPreprocessing as tp

input_file = 'assets/Almanack/Eric-Jorgenson_The-Almanack-of-Naval-Ravikant_EBOOK_v103.epub'
output_markdown = input_file.replace('.epub', '.md')

# input_file = 'Olthaus/18_Olthaus_Master.pdf'
# output_markdown = input_file.replace('.pdf', '.md')

# Check if the input file exists
if not os.path.exists(input_file):
    print(f"Input file {input_file} not found. Please check the path.")
else:
    print(f"Found input file: {input_file}")

converted_path = tp.convert_book_to_markdown(input_file, output_markdown)
print(f"Markdown version saved to: {converted_path}")

# Cleaning markdown text
markdown = ""
with open(output_markdown, 'r', encoding='utf-8') as file:
    markdown = file.read()
if not markdown:
    print("Markdown file is empty. Please check the conversion process.")

cleaned_markdown = tp.clean_markdown_text(markdown)

# Save the cleaned markdown text to a new file
cleaned_markdown_path = converted_path.replace('.md', '_cleaned.md')
with open(cleaned_markdown_path, 'w', encoding='utf-8') as file:
    file.write(cleaned_markdown)
print(f"Cleaned markdown version saved to: {cleaned_markdown_path}")

Found input file: Almanack/Eric-Jorgenson_The-Almanack-of-Naval-Ravikant_EBOOK_v103.epub
Markdown version saved to: Almanack/Eric-Jorgenson_The-Almanack-of-Naval-Ravikant_EBOOK_v103.md
Cleaned markdown version saved to: Almanack/Eric-Jorgenson_The-Almanack-of-Naval-Ravikant_EBOOK_v103_cleaned.md


/home/renlephy/miniconda3/envs/python-api/lib/python3.12/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/home/renlephy/miniconda3/envs/python-api/lib/python3.12/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


# Step 2: Text Segmentation
The text is then segmented into smaller parts for easier processing of the embeddings. The segmentation is done using the `textSegmentation` module. Two methods are provided for segmentation: 
1. **NLTK**: This method uses the Natural Language Toolkit (NLTK) for text segmentation.
2. **LangChain**: This method uses the LangChain library for text segmentation.

In [2]:
# Import the segmentation functions from your module
import textSegmentation as ts

# Load your text from a file (e.g., your Markdown version of the book)
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

book_text = cleaned_markdown

method = 'NLTK'
# method = 'LangChain'

segments = []

if method == 'NLTK':
    print("Using NLTK for text segmentation.")

    # Option 1: Segment using NLTK's TextTiling
    segments = ts.segment_text_texttiling(book_text, w=5, k=2)
    print("NLTK TextTiling produced", len(segments), "segments.")

if method == 'LangChain':
    print("Using LangChain for text segmentation.")
    
    # Option 2: Segment using LangChain's splitter
    try:
        segments = ts.segment_text_langchain(book_text, chunk_size=512, chunk_overlap=50)
        print("LangChain splitter produced", len(segments), "chunks.")
    except ImportError as e:
        print("LangChain not installed. Please install it via 'pip install langchain'.")


# Save the segments to a file
ts.save_segments_to_file(segments, input_file.replace('.epub', '_segments.txt'))

Using NLTK for text segmentation.
NLTK TextTiling produced 914 segments.


# Step 3: Embedding Generation
The segmented text is then converted into embeddings using the Sentence Transformers library. 

In [3]:
import logging
from sentence_transformers import SentenceTransformer

# Set logging to INFO level so that you can see more details in console output
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Encode with the progress bar enabled
embeddings = model.encode(segments, show_progress_bar=True, batch_size=32)

2025-04-10 09:34:47,547 - INFO - Use pytorch device_name: cuda:0
2025-04-10 09:34:47,547 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Batches:   0%|          | 0/29 [00:00<?, ?it/s]

# Step 4: Post Processing


In [4]:
import textPostprocessing as tpost

mappedEmbeddingOutput = input_file.replace('.epub', '_mapped_embeddings.json')
tpost.map_text_to_embeddings(segments, embeddings, mappedEmbeddingOutput)



Mapping of text segments to embeddings saved to: Almanack/Eric-Jorgenson_The-Almanack-of-Naval-Ravikant_EBOOK_v103_mapped_embeddings.json


# Step 5: Querying the Embeddings

In [7]:
import textQuerying as tq

# Specify the path to your mapped embedding file.
mapping_file = mappedEmbeddingOutput

# Define a query string.
query = "loves reading. Educated and uneducated."

# Load your SentenceTransformer model (should be the same model used in step 3).
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Retrieve the top 5 matching segments.
top_matches = tq.query_mapped_embeddings(query, mapping_file, model, top_k=5)

# Print out the top matching segments
for match in top_matches:
    print(f"Segment ID: {match['id']}")
    print(f"Similarity: {match['similarity']:.4f}")
    print("Text snippet:", match['text'], "...")
    print("-" * 80)

2025-04-10 09:45:07,470 - INFO - Use pytorch device_name: cuda:0
2025-04-10 09:45:07,470 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Segment ID: 407
Similarity: 0.5959
Text snippet: Reading was my first love. [4]

I remember my grandparents’ house in India. I’d be a little kid on the floor going through all of my grandfather’s Reader’s Digests, which is all he had to read. Now, of course, there’s a smorgasbord of information out there—anybody can read anything all the time. Back then, it was much more limited. I would read comic books, storybooks, whatever I could get my hands on. ...
--------------------------------------------------------------------------------
Segment ID: 408
Similarity: 0.5905
Text snippet: I think I always loved to read because I’m actually an antisocial introvert. I was lost in the world of words and ideas from an early age. I think some of it comes from the happy circumstance that when I was young, nobody forced me to read certain things. ...
--------------------------------------------------------------------------------
Segment ID: 424
Similarity: 0.5731
Text snippet: Explain what you lear